In [1]:
import pandas as pd
import numpy as np

In [38]:
def prepare_Xy():
    df = pd.read_csv('dataset.csv')
    df = df.query('CLASS_ID != 9000')
    X = df['uu_usl_name']
    y = df['CLASS_ID']
    return X,y
def texts_to_cv(X):
    from joblib import load
    cv = load('simple_vectorized_cv.joblib')
    sparce = cv.transform(X)
    return pd.DataFrame.sparse.from_spmatrix(sparce)
def normalize_texts(X):
    from pymorphy2 import MorphAnalyzer
    morph = MorphAnalyzer()
    return X.apply(lambda x: ' '.join([morph.parse(word.lower().strip('(.,!?:;\'"«»—-–)'))[0].normal_form for word in x.split()]))
def texts_to_vecs(X):
    from gensim.models import KeyedVectors
    model = KeyedVectors.load_word2vec_format('word_to_vec.bin', binary=True)
    def get_average_vector(words):
        vectors = []
        for word in words:
            if word in model:
                vectors.append(model[word])
            else:
                vectors.append(np.array([0.0] * 99 + [1.0]))
        return np.mean(vectors, axis=0)
    pre_X = [get_average_vector(text.split()) for text in X]
    return pd.DataFrame(pre_X, dtype=float)

In [24]:
X, y = prepare_Xy()

In [25]:
normalized_X = normalize_texts(X)

In [28]:
vecs_X = texts_to_vecs(normalized_X)
cv_X = texts_to_cv(normalized_X)

In [37]:
# scipy.sparse._csr.csr_matrix to df

In [40]:
total_X = pd.concat([vecs_X, cv_X], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,17697,17698,17699,17700,17701,17702,17703,17704,17705,17706
0,-0.191372,-0.244398,-0.168261,0.232595,0.652055,0.053336,0.187999,-0.201186,-0.614163,0.251317,...,0,0,0,0,0,0,0,0,0,0
1,-0.191372,-0.244398,-0.168261,0.232595,0.652055,0.053336,0.187999,-0.201186,-0.614163,0.251317,...,0,0,0,0,0,0,0,0,0,0
2,-0.077774,-0.288235,0.035577,0.285684,0.560462,-0.148274,0.155463,-0.072728,-0.568736,0.051853,...,0,0,0,0,0,0,0,0,0,0
3,-0.372827,-0.135439,0.156183,-0.010491,0.134077,-0.344258,-0.224037,0.029237,-0.082438,0.520093,...,0,0,0,0,0,0,0,0,0,0
4,-0.372827,-0.135439,0.156183,-0.010491,0.134077,-0.344258,-0.224037,0.029237,-0.082438,0.520093,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_X, y, test_size=0.2, random_state=42)